In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read Data

In [ ]:
df1 = pd.read_csv('../input/used-car-price-data/car_data.csv')
df2 = pd.read_csv('../input/used-car-price-data/model_data.csv')

In [ ]:
df1.head()

In [ ]:
len(df1)

In [ ]:
df2.head()

In [ ]:
len(df2)

Merge 2 tables as one dataset by column 'Model'

In [ ]:
df=df1.merge(df2,left_on='Model', right_on='Model')

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import date
t = date.today()
t_date=t.strftime("%d-%m-%Y")
y = t.strftime("%Y")

# Data Cleaning

#Find how many year old

In [ ]:
df['car_old']=df['Year'].apply(lambda x: int(y)-x)

Create price column from Current price column

In [ ]:
price=[]
rep=['Lakh','Lakhs','Rs.',',']
for i in df['Current Price']:
    if type(i) is str:
        for j in rep:
            i=i.replace(j,'')
        price.append(float(i))
    elif type(i) is float:
        price.append(i)
    
df['Price']=price      
df=df[df['Price']!=0]

In [ ]:
price=[]
for i in df['Price']:
    #print (i)
    if i//10 <= 100.0:
        price.append(round(i*100000,2))
    else:
        price.append(round(i,2))
    
df['Price']=price      


Update Insurance column

In [ ]:
insu =[]
for i in df['Insurance']:
    if (i=='Insurance Expired') | (i=='Expired'):
        insu.append('Expired')
    elif str(i) <t_date:
        insu.append('Expired')
    elif str(i) >=t_date:
        insu.append('Current')
    else:
        insu.append(i)
df['Insurance_status']=insu
df=df.drop('Insurance',axis=1)
df=df.drop('Current Price',axis=1)

In [ ]:
import re
df['company']=df['Model'].apply(lambda x: re.findall(r"[A-Z][^A-Z]*",x)[0])

df['company']=df['company'].apply(lambda x: 'Hyundai' if x =='Hyundaii20' else x)
df['company']=df['company'].apply(lambda x: 'Hyundai' if x =='Hyundaii10' else x)
df['company']=df['company'].apply(lambda x: 'BMW' if x is 'B' else x)
df['company']=df['company'].apply(lambda x: 'MGHECTORSHARPDCT' if x is 'M' else x)

In [ ]:
df.head()

In [ ]:
df['Insurance']=df['Insurance_status'].apply(lambda x: 1 if x=='Current' else 0)

In [ ]:
df.drop(labels=['Year'], axis=1, inplace=True)
df.drop('Insurance_status', axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df['Transmission'].value_counts()

Transmission columns consist of junk values other than Automatic or Manual Tansmission. lets us replace the Junk values with "MANUAL" assuming the cars are Manual Tansmission. 
1. Automatic -- 1
2. Manual --0

In [ ]:
df['Transmission']=df['Transmission'].apply(lambda x: 1 if x=='AUTOMATIC' else 0)

In [ ]:
df['Fuel Type'].value_counts()

In [ ]:
#since the CNG & LPG are less consuming fueal type, i hope we can consider the Pertol+CNG & Petrol+LPG as pertol vechile
#Diesel -- 1
#Petrol -- 0

df['Fuel Type']=df['Fuel Type'].apply(lambda x: 1 if x=='Diesel' else 0)

In [ ]:
df.head()

In [ ]:
df.dropna(axis=0, inplace=True)

# Explore the Target feature

In [ ]:
df.corr()['Selling Price'].sort_values()

In [ ]:
fig = px.scatter_3d(df, x='Selling Price', y='Car Condition', z='Price', color='Selling Price')
fig.show()

In [ ]:
sns.heatmap(df.corr(), annot=True)

In [ ]:
sns.boxplot(df['Selling Price'])

In [ ]:
df.head()

In [ ]:
num=['Selling Price','Kilometers Driven','Car Condition','car_old','Price']

In [ ]:
from scipy import stats
z_score = np.abs(stats.zscore(df[num]))
filtered = (z_score <2).all(axis=1)
df=df[filtered]

In [ ]:
df.shape

# Train Test Split

In [ ]:
X=df.drop('Selling Price', axis=1)
y=df['Selling Price']
cat=X.select_dtypes(include='object').columns
num=X.select_dtypes(exclude='object').columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,  random_state=42, shuffle=True)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
col_transfer = ColumnTransformer([('cat_col',OneHotEncoder(sparse=False),cat),('num_col',StandardScaler(),num)], remainder='passthrough')

# Column Transfer

In [ ]:
col_transfer.fit(X)
X_train=col_transfer.transform(X_train)
X_test = col_transfer.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)


In [ ]:
test_residual = y_test - pred
sns.scatterplot(x=y_test, y=test_residual)
plt.axhline(y=0,color='r',ls='--')

In [ ]:
sns.distplot(test_residual)

In [ ]:
from sklearn.linear_model import ElasticNet,Lasso,Ridge, SGDRegressor, LinearRegression
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

model = [ElasticNet(),Lasso(),Ridge(), SGDRegressor(), SVR(), RandomForestRegressor(), AdaBoostRegressor()]
for i in model[0:4]:
    gride = GridSearchCV(estimator=i, cv=10, param_grid={'alpha': [0.00001,0.0001,0.001,0.01,0.1,1,1.10, 10,100]}, scoring='r2')
    gride.fit(X_train, y_train)
    pred = gride.predict(X_test)
    print("Regression: {}\n".format(i))
    print("Best Score: {}\n".format(np.round(gride.best_score_,2)*100))
    print("Best Param: {}\n".format(gride.best_params_,2))
    print("R2 score from Predict: {}".format(np.round(r2_score(y_test, pred),2)))
    

In [ ]:
from sklearn.model_selection import cross_val_score
def get_valdiation(model):
    cv=cross_val_score(model,X_train,y_train,cv=10)
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    print("Regression: {} \n".format(model))
    print("Score: {:.2f} %".format(np.mean(cv)*100))
    print("Standard deviation: {:.2f}".format(np.std(cv)))
    print("R2 score from Predict: {} \n\n".format(np.round(r2_score(y_test, pred),2)))
    test_residual = y_test - pred
    sns.scatterplot(x=y_test, y=test_residual)
    plt.axhline(y=0,color='r',ls='--')
    plt.title("Regressor : {}".format(model))
    plt.show()
    sns.distplot(test_residual)
    plt.show()

In [ ]:
model = [ElasticNet(alpha=0.001),Lasso(alpha=10),Ridge(alpha=1), SGDRegressor(), RandomForestRegressor(), AdaBoostRegressor(), LinearRegression()]
for i in model:
    get_valdiation(i)

# Conclusion: 
RandomForest Regressor works well for this problem with 85% accuracy